# CHROMA-BASED SUPERVISED MODE RECOGNITION IN THE MULTI-CULTURAL CONTEXT


There are 3 main steps in the workflow of this notebook: Feature Extraction, Data Formatting and Classification. 
---

IMPORTANT : All the audio files should be located in the directory specifically according to their modality type.



## STEP 1 : GATHERING DATA / DOWNLOADING THE AUDIO FILES

### Input : 
#### JSON file :
which contains the mbids of audio files, the tonic information, the modality category (e.g. makam, mode, rag, tab, etc. ) In the case of multiple modalities within one song, the start and end times of different sections needs to be specified.

#### Directory :
Output directory for the dataset to be downloaded


In [1]:
from compmusic import dunya
from modeUtils.utilities import downloadDataset

#token will be shared upon request
dunya.set_token('_EnterYourTokenHere_') 

### Please set the directory for the dataset to be downloaded and the name of the annotations file (JSON).
dataDir = 'data/Turkish/'
annotationsFile = 'annotations_makam.json'

#dataDir = 'data/Hindustani/'
#annotationsFile = 'annotations_hindustani.json'

#dataDir = 'data/Carnatic/'
#annotationsFile = 'annotations_carnatic.json'

In [2]:
downloadDataset(annotationsFile,dataDir)

{'Bhairavi', 'Kāṁbhōji', 'Kalyāṇi', 'Kāpi', 'Kamās', 'Tōḍi', 'Pantuvarāḷi', 'Saurāṣtraṁ', 'Śankarābharaṇaṁ', 'Rāgamālika'}
f0f94781-bf80-4248-b475-4a2d9abc7593.mp3
b8e5c465-b4a1-45a5-ba32-824fbd4cfe3a.mp3
523a60bf-3060-4e07-9907-0435b76114be.mp3
4fb8af33-df20-48dc-ad21-0eeb7eb47c91.mp3
9ea8380c-ef54-46ee-8496-8044dac4f8e9.mp3
fcc60a0f-6ced-4a05-b7b8-4dc29d87c546.mp3
52b3552e-dc72-4995-b47d-bf9c4e6d7155.mp3
8c4fc946-e15a-4ffc-9e08-a6f5cd819b9a.mp3
2da84bf7-c3ab-415f-b498-0a3096a8c8d2.mp3
83cd0310-c7f8-4f5c-910f-ee6bba41ed3e.mp3
bc6f2cbc-abe3-4bba-b4b4-a876710b7650.mp3
0d09bf2a-5fef-43df-a50f-a0bfa92af646.mp3
5020d866-f282-41d6-b653-39f930f83c8c.mp3
c4cc6ba4-d456-4fb8-8d31-56a19a7d6312.mp3
e57d03ee-098c-4612-a1aa-988a653bfe41.mp3
49039ca4-4ece-426a-a358-70a61d8f25f2.mp3
37d61e4c-6f7e-42ec-887a-95f84e27238a.mp3
70b0dd53-2a4d-4e59-bc6d-fd92a8b1b3f2.mp3
64fe1441-a9a0-4a3f-86e6-91321958668e.mp3
ee40de57-4e17-4d70-952c-f6919f8d722c.mp3
Dataset downloaded and created in data/Carnatic/ folder


## STEP 2 : FEATURE EXTRACTION

### Input : 
 - JSON file :
    which contains the mbids of audio files, the tonic information, the modality category (e.g. makam, mode, rag, tab, etc. ) In the case of multiple modalities within one song, the start and end times of different sections needs to be specified.

 - numBins(int) : Number of bins in the Chroma Vectors. This is parametrized in consideration of possible microtonalities existing in non-Western music traditions (12,24,36,48, ...)
 - musicTradition (str) : Name of the modality type specific for the music tradition.
 
 
         Jazz : Chord-scale
 
         Turkish Classical Music : Makam
 
         Hindustani Classical Music : Rag
 
         Carnatic Classical Music : Raaga
 
         Arab-Andalusian Music : Tab

### Output : 
#### PICKLE file :
which has a list of dictionaries where each dictionary contains the Frame-based features, the global statistical features and the ground truth information (tonic, mode type) of each of the audio files.

##### IMPORTANT : All the audio files should be located in the directory specifically according to their harmonic mode type.


Note that the functions that are used in this tutorial can be found in modeUtils/utilities.py, which is provided in the repository.

In [6]:
!python3 ChromaFeatureExtraction.py -h

usage: ChromaFeatureExtraction.py [-h] -t TRADITION -n NUMBEROFBINS -o
                                  OUTPUT_DIRECTORY

A tool for Chroma (HPCP) Feature Extraction using Essentia library. (FEATURE
EXTRACTION)

optional arguments:
  -h, --help            show this help message and exit
  -t TRADITION, --tradition TRADITION
                        Input music tradition to perform the mode
                        classification task
  -n NUMBEROFBINS, --numberofBins NUMBEROFBINS
                        Input number of bins per octave in chroma vectors
  -o OUTPUT_DIRECTORY, --output_directory OUTPUT_DIRECTORY
                        Output directory for the pickle file that contains the
                        dataset with the extracted features


In [ ]:
!python3 ChromaFeatureExtraction.py -t CarnaticClassicalMusic -n 12 -o 'data/Carnatic/'

Analysis on CarnaticClassicalMusic Tradition.

Number of bins per octave in the Chroma Vectors : 12
mode categories in the dataset : 

{'Tōḍi', 'Kamās', 'Bhairavi', 'Kāpi', 'Rāgamālika', 'Kalyāṇi', 'Kāṁbhōji', 'Kāmavardani/Pantuvarāḷi', 'Śankarābharaṇaṁ', 'Saurāṣtraṁ'} 

Number of Categories in the dataset
10
Feature Extraction in Process. This might take a while...
extracting Features for Raaga :  Saurāṣtraṁ
extracting Features for Raaga :  Tōḍi


In [43]:
!python3 DataFormatting.py -h

usage: DataFormatting.py [-h] -i INPUT_FILENAME -f FEATURES_SET -r REGION -c
                         COMBINED

A tool for converting the feature data into proper format for the machine
learning pipeline

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT_FILENAME, --input_filename INPUT_FILENAME
                        Name of the input pickle file which is the output file
                        of ChromaFeatureExtraction.py
  -f FEATURES_SET, --features_set FEATURES_SET
                        Set of features to include in the Feature Data.
                        options = {"mean","std","all"}
  -r REGION, --region REGION
                        Input region of audios (from the beginning) for
                        extracting local features. values = [0,1] (region = 1
                        is equivalent of using the Global Features)
  -c COMBINED, --combined COMBINED
                        Input number of bins per octave in chroma vectors SET

## STEP 3 :  MACHINE LEARNING & AUTOMATIC AUDIO CLASSIFICATION

### Input : 
#### PICKLE file : 
which is the output of the first notebook in the series (Feature Extraction).


### Output :
#### CSV file : 
The output CSV file that contains the FeatureData (X) and the class labels (Y), in appropriate format for the Machine Learning Pipeline.

#### Parameters:


 - numBins(int) : number of bins in the HPCPvectors ([12,24,36,48])
 
 
 - region(float) : select the first 'x*100'% of the audio files for analysis. The classification is performed using the features that are extracted locally only from specified region of the songs.
 

 - combined (boolean) : if combined = 1, classification is performed using the combination of features that are extracted both locally and globally. if combined = 0, only locally obtained features are used for classification. The local region is specified by partSong. Default = True.
 

In [19]:
!python3 DataFormatting.py -i 'extractedFeatures_forRagtradition(12bins).pkl' -r 0.3 -c True

Rag
computing Local Features for the first0.3region of the audio files is COMPLETE 

Mode types in this dataset:  

{'Khamaj', 'Lalat', 'Todi', 'Bageshree', 'Des', 'Bilaskhani todi', 'Yaman kalyan', 'Malkauns', 'Marwa', 'Bhairabi'}
Generating CSV file for the features meanLocal+stdGlobal is COMPLETE
Generating CSV file for the features stdLocal+meanGlobal is COMPLETE
Generating CSV file for the features meanLocal+meanGlobal is COMPLETE
Generating CSV file for the features stdLocal+stdGlobal is COMPLETE


In [17]:
!python3  Classification.py -h

usage: Classification.py [-h] -i INPUT_FILENAME -c COMBINED -r REGION

A tool for Chroma (HPCP) Feature Extraction using Essentia
library.(CLASSIFICATION)

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT_FILENAME, --input_filename INPUT_FILENAME
                        Name of the input pickle file which is the output file
                        of ChromaFeatureExtraction.py
  -c COMBINED, --combined COMBINED
                        Input number of bins per octave in chroma vectors SET
                        combined = 1 FOR PERFORMING CLASSIFICATION USING THE
                        COMBINATION OF LOCAL AND GLOBAL FEATURES; combined = 0
                        FOR USING ONLY THE LOCAL FEATURES (except the case
                        region = 1, which ALREADY corresponds to global
                        features)
  -r REGION, --region REGION
                        Input region of audios (from the beginning) for
                        extract

In [20]:
###CHOOSE THE CSV FILE WITH DESIRED FEATURE SET, TO INPUT FOR THE MACHINE LEARNING PIPELINE
import os
for file in os.listdir(dataDir):
    if file.endswith('.csv'):
        print(file)

DataCSVforstage_12bins_meanLocal+meanGlobal.csv
DataCSVforstage_12bins_meanLocal+stdGlobal.csv
DataCSVforstage_12bins_stdLocal+stdGlobal.csv
DataCSVforstage_12bins_stdLocal+meanGlobal.csv


In [22]:

!python3 Classification.py -i 'DataCSVforstage_12bins_stdLocal+stdGlobal.csv' -m Rag -r 0.3

This process might take a while (5-10 min) 
 CROSS-VALIDATION & TRAINING 
Accuracy score for the Feature Set stdLocal+stdGlobal : 
F-measure (mean,std) --- FINAL
0.82 0.0749427597141
Accuracy (mean,std) FINAL
0.82 0.0715891053164
['Bageshree', 'Bhairabi', 'Bilaskhani todi', 'Des', 'Khamaj', 'Lalat', 'Malkauns', 'Marwa', 'Todi', 'Yaman kalyan']
Confusion matrix, without normalization
Figure(900x1000)
